<a id="eight"></a>
##  <p style="background-color:#070B19;font-family:TimesNewRoman;color:#81F7F3;font-size:90%;text-align:left;border-radius:15px 15px;padding:15px"> Kafka Stocks Data Load ~ Bronze</p>

In [0]:
!pip install pyspark

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 726.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 40.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741244 sha256=436cc58dc9c96e38ff6010720e52d4f32e5b7e4adabc585f08dadb71a5981de4
  Stored in directory: /root/.cache/pip/wheels/ec/3d/2d/acbccedd6a65f607dc86a5d4615f54598cae855a943030d2b1
Successfully built pyspark
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [0]:
spark = SparkSession.builder.appName("KafkaStockData").getOrCreate()

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

DataFrame[]

In [0]:
spark.sql("DROP TABLE IF EXISTS bronze.stocks_fx_crypto_data")

DataFrame[]

In [0]:
# Define tables for raw stock data in the Bronze layer
spark.sql("""
    CREATE TABLE IF NOT EXISTS bronze.stocks_fx_crypto_data (
        ticker STRING,
        trade_volume DECIMAL,
        volume_weighted_avg_price DECIMAL,
        open DECIMAL,
	    close DECIMAL,
	    high DECIMAL,
	    low DECIMAL,
		timestamp_ms BIGINT,
	    no_of_trans_aggr_window BIGINT 
     )
    USING PARQUET
""")

DataFrame[]

#####  <p style="background-color:#070B19;font-family:TimesNewRoman;color:#81F7F3;font-size:100%;text-align:left;border-radius:15px 15px;padding:15px"> Predefine Stocks DataFrame Schema </p>

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, DoubleType, TimestampType, LongType

schema = StructType([
    StructField("ticker", StringType(), True),
    StructField("v", DecimalType(18, 2), True),
    StructField("vw", DecimalType(18, 2), True),
    StructField("o", DecimalType(18, 2), True),
    StructField("c", DecimalType(18, 2), True),
    StructField("h", DecimalType(18, 2), True),
    StructField("l", DecimalType(18, 2), True),
    StructField("t", LongType(), True),
    StructField("n", LongType(), True)
])



In [0]:
from pyspark.sql.functions import col, from_unixtime

# File location and type
file_location = "/FileStore/tables/kafka_output_20250608_054406.csv"
file_type = "csv"

# CSV options
first_row_is_header = "true"
delimiter = ","

# Load the data using the predefined schema
bronze_df = spark.read.format(file_type) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .schema(schema) \
    .load(file_location)

display(bronze_df)
bronze_df = bronze_df.withColumn("t", from_unixtime(col("t") / 1000))
bronze_df.show(5)


ticker,v,vw,o,c,h,l,t,n
fx_CAUDZAR,424629486.00,177.13,192.24,182.01,192.87,159.70,1711944000000,5725103
fx_CAUDZAR,375625499.00,172.40,182.20,154.59,196.95,154.02,1719806400000,5253722
stocks_BA,8951574.00,182.73,178.09,184.57,186.01,178.02,1717387200000,115959
stocks_BA,7176817.00,186.62,185.05,188.62,188.84,183.25,1717473600000,99850
stocks_BA,5943952.00,188.95,187.90,189.85,190.34,186.53,1717560000000,85202
stocks_BA,5942160.00,190.30,188.83,191.42,192.10,187.35,1717646400000,77577
stocks_BA,3374548.00,190.59,190.46,190.24,191.98,189.66,1717732800000,59599
stocks_BA,4388254.00,190.53,190.00,190.12,192.63,189.60,1717992000000,63572
stocks_BA,5010058.00,185.29,189.32,185.50,189.32,182.75,1718078400000,77065
stocks_BA,5245156.00,183.36,186.50,182.67,187.16,181.71,1718164800000,80797


+----------+------------+------+------+------+------+------+-------------------+-------+
|    ticker|           v|    vw|     o|     c|     h|     l|                  t|      n|
+----------+------------+------+------+------+------+------+-------------------+-------+
|fx_CAUDZAR|424629486.00|177.13|192.24|182.01|192.87|159.70|2024-04-01 04:00:00|5725103|
|fx_CAUDZAR|375625499.00|172.40|182.20|154.59|196.95|154.02|2024-07-01 04:00:00|5253722|
| stocks_BA|  8951574.00|182.73|178.09|184.57|186.01|178.02|2024-06-03 04:00:00| 115959|
| stocks_BA|  7176817.00|186.62|185.05|188.62|188.84|183.25|2024-06-04 04:00:00|  99850|
| stocks_BA|  5943952.00|188.95|187.90|189.85|190.34|186.53|2024-06-05 04:00:00|  85202|
+----------+------------+------+------+------+------+------+-------------------+-------+
only showing top 5 rows



In [0]:
# Rename multiple columns using selectExpr()
bronze_df_renamed = bronze_df.selectExpr(
    "ticker as ticker",
    "v as trade_volume",
    "vw as volume_weighted_avg_price",
    "o as open_price",
    "c as close_price",
    "h as highest_price",
    "l as lowest_price",
    "t as timestamp_sec",
    "n as no_of_trans_aggr_window"
)

# Show DataFrame with new column names
display(bronze_df_renamed)


ticker,trade_volume,volume_weighted_avg_price,open_price,close_price,highest_price,lowest_price,timestamp_sec,no_of_trans_aggr_window
fx_CAUDZAR,424629486.00,177.13,192.24,182.01,192.87,159.70,2024-04-01 04:00:00,5725103
fx_CAUDZAR,375625499.00,172.40,182.20,154.59,196.95,154.02,2024-07-01 04:00:00,5253722
stocks_BA,8951574.00,182.73,178.09,184.57,186.01,178.02,2024-06-03 04:00:00,115959
stocks_BA,7176817.00,186.62,185.05,188.62,188.84,183.25,2024-06-04 04:00:00,99850
stocks_BA,5943952.00,188.95,187.90,189.85,190.34,186.53,2024-06-05 04:00:00,85202
stocks_BA,5942160.00,190.30,188.83,191.42,192.10,187.35,2024-06-06 04:00:00,77577
stocks_BA,3374548.00,190.59,190.46,190.24,191.98,189.66,2024-06-07 04:00:00,59599
stocks_BA,4388254.00,190.53,190.00,190.12,192.63,189.60,2024-06-10 04:00:00,63572
stocks_BA,5010058.00,185.29,189.32,185.50,189.32,182.75,2024-06-11 04:00:00,77065
stocks_BA,5245156.00,183.36,186.50,182.67,187.16,181.71,2024-06-12 04:00:00,80797


#####  <p style="background-color:#070B19;font-family:TimesNewRoman;color:#81F7F3;font-size:100%;text-align:left;border-radius:15px 15px;padding:15px"> Check for any missing values per column </p>

In [0]:
from pyspark.sql import functions as F

# Compute percentage of missing values per column
display(bronze_df_renamed.select([(F.count(F.when(F.col(c).isNull(), c)) / F.count("*")).alias(c) for c in bronze_df_renamed.columns]))

ticker,trade_volume,volume_weighted_avg_price,open_price,close_price,highest_price,lowest_price,timestamp_sec,no_of_trans_aggr_window
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#####  <p style="background-color:#070B19;font-family:TimesNewRoman;color:#81F7F3;font-size:100%;text-align:left;border-radius:15px 15px;padding:15px"> Save DataFrame to CSV file </p>

In [0]:
# Save the cleaned DataFrame to CSV
bronze_df_renamed.write.mode("overwrite").option("header", "true").csv("/FileStore/tables/silver_kafka_output_20250608_054406.csv")